# Introduction

This notebook presents the several machine learning models using CNN and LSTM for HAR. To obtain a detailed description of the architecture, please refer to the dissertation, **"RECOGNISING HUMAN ACTIVITIES AUTONOMOUSLY THROUGH FUSION OF SENSOR DATA"**.

## Dataset

As a dataset, the [Human Activity Recognition Using Smartphones Data Set](https://archive.ics.uci.edu/ml/datasets/Human+Activity+Recognition+Using+Smartphones) is used. To prepare this dataset for the program, the following code is uncommented and executed.

In [ ]:
# Download dataset zip file and place data files in training and test set directries

zipfile_dataset_uci_har = "UCI HAR Dataset.zip"
url_dataset_uci_har = "https://archive.ics.uci.edu/ml/machine-learning-databases/00240/UCI%20HAR%20Dataset.zip"

#!wget $url_dataset_uci_har
#!unzip "$zipfile_dataset_uci_har"
#!ls "UCI HAR Dataset"

# 1.Parameters

Adjustable flags and parameters are listed. Hyperparameters for each ML model are in "[F] ML models" section.

|Name|Type|Explanation|
|-|-|-|
|flag_(ML model name)|Flag|Whether execute the model or not|
|flag_experiment|Flag|Whether run repeated evaluation for summary statistics or not|
|flag_model_load|Flag|Whether load the model from the file or not|
|flag_model_save|Flag|Whether save the model to the file after training or not|
|flag_EarlyStopping|Flag|Enable Early stopping|
|flag_es_monitor|Flag|Monitor type for Early stopping|
|ratio_train|Parameter|The ratio between training and validation sets|
|seed|Parameter|Fix the seed for reproducibility|
|flag_scale|Flag|Scaling technique|
|flag_data_dist|Flag|Display dataset distribution|
|flag_plot_model|Flag|Whether plot model graphs or not|
|flag_save_fig|Flag|Whether save graphs or not|
|flag_summary|Flag|Show summary of the dataset|
|flag_cm_norm|Flag|Whether normalise confusion matrix or not|
|flag_TensorBoard|Flag|Whether save the Tensorboard data or not|

In [ ]:
### [Note]
#
# Hyperparameters for each ML model are in "[F] ML models" section
#
### ---------- ---------- ---------- ---------- ----------
### Flags

# ML
flag_CNN_1d         = True
flag_LSTM_Mto1      = True
flag_CNN1D_LSTM     = True
flag_ConvLSTM       = True
flag_Ensemble       = True

flag_experiment     = True

flag_model_load     = False
flag_model_save     = True

flag_EarlyStopping  = True
flag_es_monitor     = "val_loss"
#flag_es_monitor     = "val_accuracy"
### ---------- ---------- ---------- ---------- ----------
### Pre-processing
# Ratio of training dataset to be split
ratio_train = 0.85

# Randam seed for reproducibility
seed = 7

# scaling
flag_scaling        = "Std" # for Gaussian
#flag_scaling        = "Norm" # (0 - 1)
### ---------- ---------- ---------- ---------- ----------
### Evaluation
flag_data_dist      = False
flag_plot_model     = True
flag_savefig        = True

flag_summary        = True
flag_cm_norm        = True

flag_TensorBoard    = False
### ---------- ---------- ---------- ---------- ----------
### Directories
dir_log = 'log'
dir_model = 'model'
### ---------- ---------- ---------- ---------- ----------
### Names
# models
modelname_cnn_1d         = 'CNN_1D'
modelname_lstm_Mto1      = 'LSTM_Mto1'
modelname_lstm_MtoM      = 'LSTM_MtoM'
modelname_cnn1d_lstm     = 'CNN1D_LSTM'
modelname_convlstm       = 'ConvLSTM'
modelname_ensemble       = 'Ensemble'
modelname_lstm_Mto1_null = 'LSTM_Mto1_null'

# Label list
labels_activity = ['WALKING',
                   'UPSTAIRS',
                   'DOWNSTAIRS',
                   'SITTING',
                   'STANDING',
                   'LAYING']
### ---------- ---------- ---------- ---------- ----------

# 2.Setup

## Import libraries

In [ ]:
# Pre-process
import os
import glob
import numpy as np
import random as rn
import pandas as pd
from numpy import dstack
from pandas import read_csv
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from tensorflow.keras.utils import to_categorical

# Evaluation
from sklearn.metrics import f1_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import time
from datetime import datetime
from numpy import mean, std
from matplotlib import pyplot
import seaborn as sns
sns.set()

# NNs
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard

# CNN
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv1D, MaxPool1D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.keras.layers import Input, BatchNormalization
from tensorflow.keras.optimizers import Adam

# LSTM
from tensorflow.keras.layers import LSTM, TimeDistributed
from tensorflow.keras import regularizers

# ConvLSTM
from tensorflow.keras.layers import ConvLSTM2D

# Ensemble
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Concatenate

In [ ]:
# Set random seed (for reproducibility)
# Hash seed
os.environ["PYTHONHASHSEED"] = str(seed)

# Built-in random
rn.seed(seed)

# Numpy.random
np.random.seed(seed)

# Tensorflow
tf.random.set_seed(seed)

## [F] Pre-processing

In [ ]:
# load a single file as a numpy array
def load_file(filepath):
	dataframe = read_csv(filepath, header=None, delim_whitespace=True)
	return dataframe.values
 
# load a list of files and return as a 3d numpy array
def load_group(filenames, prefix=''):
	loaded = list()
	for name in filenames:
		data = load_file(prefix + name)
		loaded.append(data)

	# stack group so that features are the 3rd dimension
	loaded = dstack(loaded)

	return loaded
 
# load a dataset group, such as train or test
def load_dataset_group(group, prefix=''):
	filepath = prefix + group + '/Inertial Signals/'

	# load all 9 files as a single array
	filenames = list()

	# total acceleration
	filenames += ['total_acc_x_'+group+'.txt', 'total_acc_y_'+group+'.txt', 'total_acc_z_'+group+'.txt']

	# body acceleration
	filenames += ['body_acc_x_'+group+'.txt', 'body_acc_y_'+group+'.txt', 'body_acc_z_'+group+'.txt']

	# body gyroscope
	filenames += ['body_gyro_x_'+group+'.txt', 'body_gyro_y_'+group+'.txt', 'body_gyro_z_'+group+'.txt']

	# load input data
	X = load_group(filenames, filepath)

	# load class output
	y = load_file(prefix + group + '/y_'+group+'.txt')

	return X, y
 
# load the dataset, returns train and test X and y elements
def load_dataset(prefix=''):
	# load all train
	trainX, trainy = load_dataset_group('train', prefix + 'UCI HAR Dataset/')
	print(f'Train: {trainX.shape} {trainy.shape}')

	# load all test
	testX, testy = load_dataset_group('test', prefix + 'UCI HAR Dataset/')
	print(f'Test: {testX.shape} {testy.shape}')

	# zero-offset class values
	trainy = trainy - 1
	testy = testy - 1

	# one hot encode y
	trainy = to_categorical(trainy)
	testy = to_categorical(testy)

	return trainX, trainy, testX, testy

## [F] ML models

In [ ]:
# Train
epochs = 100
batch_size = 64
repeats = 10

# EarlyStopping
es_patience = 5

### CNN

In [ ]:
# Layer
cnn_padding ='same'
cnn_activation = 'relu'
cnn_units = 128
cnn_dropout = 0.5
cnn_pool_size = 2
cnn_weight_decay = 1e-4

## 1D Conv
cnn_1d_filters = 64
cnn_1d_kernel_size = 5

In [ ]:
def build_model_cnn_1d():
    model = Sequential(name=modelname_cnn_1d)

    # Conv layer 1
    model.add(Conv1D(
        input_shape = cnn_1d_input_shape,
        filters     = cnn_1d_filters,
        kernel_size = cnn_1d_kernel_size,
        padding     = cnn_padding,
        activation  = cnn_activation))

    # Conv layer 2
    model.add(Conv1D(
        filters     = cnn_1d_filters,
        kernel_size = cnn_1d_kernel_size,
        padding     = cnn_padding,
        activation  = cnn_activation))

    # Conv layer 3
    model.add(Conv1D(
        filters     = cnn_1d_filters,
        kernel_size = cnn_1d_kernel_size,
        padding     = cnn_padding,
        activation  = cnn_activation))

    # Conv layer 4
    model.add(Conv1D(
        filters     = cnn_1d_filters,
        kernel_size = cnn_1d_kernel_size,
        padding     = cnn_padding,
        activation  = cnn_activation))

    # Maxpool layer
    # model.add(MaxPool1D(
    #     pool_size = cnn_pool_size))

    model.add(Flatten())

    # Dense layer 1
    model.add(Dense(
        units      = cnn_units,
        activation = 'relu'))

    # Dropout
    model.add(Dropout(cnn_dropout))

    # Dense layer 2
    model.add(Dense(
        units      = cnn_units,
        activation = 'relu'))

    # Dropout
    model.add(Dropout(cnn_dropout))

    # Output layer
    model.add(Dense(
        units      = num_classes,
        activation = 'softmax'))

    model.compile(
        loss       = 'categorical_crossentropy',
        optimizer  = 'adam',
        metrics    = ['accuracy'])

    return model

### LSTM

In [ ]:
# Layer
lstm_units = 128
lstm_dropout = 0.5
lstm_weight_decay = 1e-4

In [ ]:
# LSTM (Many-to-One, stateless)
def build_model_lstm_Mto1():
    model = Sequential(name=modelname_lstm_Mto1)

    # LSTM layer
    model.add(LSTM(
        input_shape        = lstm_input_shape,
        units              = lstm_units,
        # kernel_regularizer = regularizers.l2(lstm_weight_decay),
        return_sequences   = False)) # final layer of LSTM (only final output)

    # Dropout
    model.add(Dropout(lstm_dropout))

    # Dense layer
    model.add(Dense(
        units      = lstm_units,
        activation = 'relu'))

    # Dropout
    #model.add(Dropout(lstm_dropout))

    # Output layer
    model.add(Dense(
        units      = num_classes,
        activation = 'softmax'))

    model.compile(
        loss       = 'categorical_crossentropy',
        optimizer  = 'adam',
        metrics    = ['accuracy'])

    return model

### CNN-LSTM

In [ ]:
# Data
cnn_lstm_steps = 4

# Layer
cnn_lstm_padding = 'same'
cnn_lstm_activation = 'relu'
cnn_lstm_dropout = 0.5
cnn_lstm_pool_size = 2

## CNN
cnn_lstm_filters = 64
cnn1d_lstm_kernel_size = 3

# LSTM
cnn_lstm_units = 128

In [ ]:
def build_model_cnn1d_lstm():
    model = Sequential(name=modelname_cnn1d_lstm)

    ## CNN (with TimeDistributed)
    # Conv layer 1
    model.add(TimeDistributed(Conv1D(
        filters     = cnn_lstm_filters,
        kernel_size = cnn1d_lstm_kernel_size,
        padding     = cnn_lstm_padding,
        activation  = cnn_lstm_activation),
        input_shape = cnn1d_lstm_input_shape))

    # Conv layer 2
    model.add(TimeDistributed(Conv1D(
        filters     = cnn_lstm_filters,
        kernel_size = cnn1d_lstm_kernel_size,
        padding     = cnn_lstm_padding,
        activation  = cnn_lstm_activation)))

    # Conv layer 3
    model.add(TimeDistributed(Conv1D(
        filters     = cnn_lstm_filters,
        kernel_size = cnn1d_lstm_kernel_size,
        padding     = cnn_lstm_padding,
        activation  = cnn_lstm_activation)))

    # Conv layer 4
    model.add(TimeDistributed(Conv1D(
        filters     = cnn_lstm_filters,
        kernel_size = cnn1d_lstm_kernel_size,
        padding     = cnn_lstm_padding,
        activation  = cnn_lstm_activation)))

    # Dropout
    model.add(TimeDistributed(Dropout(cnn_lstm_dropout)))

    # Maxpool layer
    model.add(TimeDistributed(MaxPool1D(
        pool_size = cnn_lstm_pool_size)))

    model.add(TimeDistributed(Flatten()))

    ## LSTM
    # LSTM layer 1
    model.add(LSTM(
        units            = cnn_lstm_units,
        return_sequences = True))

    # Dropout
    model.add(Dropout(cnn_lstm_dropout))

    # LSTM layer 2
    model.add(LSTM(
        units            = cnn_lstm_units,
        return_sequences = False))

    # Dropout
    model.add(Dropout(cnn_lstm_dropout))

    # Output layer
    model.add(Dense(
        units      = num_classes,
        activation = 'softmax'))

    model.compile(
        loss       = 'categorical_crossentropy',
        optimizer  = 'adam',
        metrics    = ['accuracy'])

    return model

### ConvLSTM

In [ ]:
# Data
convlstm_steps = 4

# Layer
convlstm_padding = 'same'
convlstm_activation = 'relu'
convlstm_dropout = 0.5
convlstm_pool_size = 2

## CNN
convlstm_filters = 64
convlstm_kernel_size = (1, 3)

convlstm_units = 128

In [ ]:
def build_model_convlstm():
    model = Sequential(name=modelname_convlstm)

    # Conv LSTM layer 1
    model.add(ConvLSTM2D(
        filters          = convlstm_filters,
        kernel_size      = convlstm_kernel_size,
        padding          = convlstm_padding,
        activation       = convlstm_activation,
        input_shape      = convlstm_input_shape,
        return_sequences = True))
        # return_sequences = False)) # final layer of LSTM (only final output)

    # Conv LSTM layer 2
    model.add(ConvLSTM2D(
        filters          = convlstm_filters,
        kernel_size      = convlstm_kernel_size,
        padding          = convlstm_padding,
        activation       = convlstm_activation,
        return_sequences = True))

    # Conv LSTM layer 3
    model.add(ConvLSTM2D(
        filters          = convlstm_filters,
        kernel_size      = convlstm_kernel_size,
        padding          = convlstm_padding,
        activation       = convlstm_activation,
        return_sequences = True))

    # Conv LSTM layer 4
    model.add(ConvLSTM2D(
        filters          = convlstm_filters,
        kernel_size      = convlstm_kernel_size,
        padding          = convlstm_padding,
        activation       = convlstm_activation,
        return_sequences = False))

    # Dropout
    model.add(Dropout(convlstm_dropout))

    model.add(Flatten())

    # Dense layer
    model.add(Dense(
        units      = convlstm_units,
        activation = convlstm_activation))

    # Output layer
    model.add(Dense(
        units      = num_classes,
        activation = 'softmax'))

    model.compile(
        loss       = 'categorical_crossentropy',
        optimizer  = 'adam',
        metrics    = ['accuracy'])

    return model

### Ensemble

In [ ]:
# Layer
ensemble_units = 10
ensemble_activation = 'relu'

In [ ]:
def build_model_ensemble(inputs, outputs):
    ensemble_merge = Concatenate(axis=1)(outputs)

    # Dense layer
    ensemble_hidden = Dense(
        units      = ensemble_units,
        activation = ensemble_activation)(ensemble_merge)

    # Output layer
    ensemble_output = Dense(
        units      = num_classes,
        activation = 'softmax')(ensemble_hidden)

    model = Model(
        inputs     = inputs,
        outputs    = ensemble_output,
        name       = modelname_ensemble)

    model.compile(
        loss       = 'categorical_crossentropy',
        optimizer  = 'adam',
        metrics    = ['accuracy'])

    return model

## [F] Evaluation

In [ ]:
# Train and evaluate a model (once)
def evaluate_model(model_name, X_train, y_train, X_val, y_val, X_test, y_test):
    # Build model
    if model_name == modelname_cnn_1d:
        model = build_model_cnn_1d()
    elif model_name == modelname_lstm_Mto1:
        model = build_model_lstm_Mto1()
    elif model_name == modelname_lstm_MtoM:
        model = build_model_lstm_MtoM()
    elif model_name == modelname_cnn1d_lstm:
        model = build_model_cnn1d_lstm()
    elif model_name == modelname_convlstm:
        model = build_model_convlstm()
    else:
        print("Error: specify correct model name")
        return -1
    
    # Train
    history = model.fit(
        x               = X_train,
        y               = y_train,
        batch_size      = batch_size,
        epochs          = epochs,
        verbose         = 0,
        callbacks       = [cb],
        validation_data = (X_val, y_val)
    )

    num_epochs = len(history.history['loss'])

    ## Evaluate
    # Accuracy
    _, accuracy = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=0)

    # F1
    if model_name == modelname_lstm_MtoM:
        y_pred = model.predict(X_test, batch_size=batch_size)
        y_pred_last = np.asarray([[i[-1]] for i in y_pred]).reshape(-1, y_pred.shape[-1])
        y_pred_last_classes = y_pred_last.argmax(axis=-1)
        f1 = f1_score(y_test_classes_lstm_MtoM, y_pred_last_classes, average='weighted')
    else:
        y_pred = model.predict(X_test)
        f1 = f1_score(y_test.argmax(axis=-1), y_pred.argmax(axis=-1), average='weighted')

    return accuracy, f1, num_epochs

In [ ]:
# Repeat experiment
def run_experiment(model_name, X_train, X_val, X_test, y_train, y_val, y_test, repeats=10):
    print(f'Model: {model_name}')

    scores_acc = []
    scores_f1 = []
    scores_epoch = []
    for r in range(repeats):
        acc, f1, epoch = evaluate_model(model_name, X_train, y_train, X_val, y_val, X_test, y_test)
        print(f'[#{r+1:>2d}] Accuracy: {acc:.4f}, F1 score(weighted): {f1:.4f}, epoch: {epoch}')
        scores_acc.append(acc)
        scores_f1.append(f1)
        scores_epoch.append(epoch)
    
    # Summarise mean and standard deviation
    print(f'Accuracy: {mean(scores_acc):.4f} (+/- {std(scores_acc):.4f})')
    print(f'F1 score(weighted): {mean(scores_f1):.4f} (+/- {std(scores_f1):.4f})')
    print(f'epoch: {mean(scores_epoch):.1f} (+/- {std(scores_epoch):.4f})')

	# Boxplot of scores
    metrics_list = ['Accuracy', 'F1 score']
    all_scores = []
    all_scores.append(scores_acc)
    all_scores.append(scores_f1)
    plt.boxplot(all_scores, labels=metrics_list)

    if flag_savefig:
        plt.savefig("boxplot_" + model_name + ".png")
        
    plt.show()

In [ ]:
# Plot a histogram of each variable in the dataset
def plot_variable_distributions(X, start=0, end=None, xlim=None):
    if end is None:
        end = X.shape[1]-1

    print(X.shape)
    num_features = end - start +1
    print(f'# of plots: {num_features} ({start} - {end})')

    plt.figure(figsize=(5, 2*num_features), tight_layout=True)
    xaxis = None
    for i, f in enumerate(range(start, end+1)):
        if xlim is None:
            ax = plt.subplot(num_features, 1, i+1, title='Feature: ' + str(f+1))
        else:
            ax = plt.subplot(num_features, 1, i+1, sharex=xaxis, title='Feature: ' + str(f+1))
            ax.set_xlim(xlim)
        if i == 0:
            xaxis = ax
        plt.hist(X[:, f], bins=100)
    plt.show()

In [ ]:
# Plot graphs for loss and accuracy
def plot_acc_graph(history):

    # Set figure size
    fig = plt.figure(figsize=(15, 6))
    plt.subplots_adjust(wspace=0.2)

    # Loss
    plt.subplot(1,2,1)
    plt.plot(history.history['loss'],
            label='Train',
            color='black')

    plt.plot(history.history['val_loss'],
            label='Val',
            color='red')

    #plt.ylim(0, 1)
    plt.legend()
    plt.grid(True)
    plt.xlabel('Epoch')
    plt.ylabel('Loss')

    # Accuracy
    plt.subplot(1,2,2)
    plt.plot(history.history['accuracy'],
            label='Train',
            color='black')

    plt.plot(history.history['val_accuracy'],
            label='Val',
            color='red')

    plt.ylim(0, 1)
    plt.legend()
    plt.grid(True)
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')

    if flag_savefig:
    	plt.savefig("acc_graph_" + history.model.name + ".png")

    plt.show()

In [ ]:
# Print execution time
def print_execution_time(time_start):
    time_elapsed = time.perf_counter() - time_start
    min, sec = divmod(time_elapsed, 60)
    hour, min = divmod(min, 60)

    print(f"Execution time: {hour:.0f} hour {min:.0f} min {sec:.0f} sec")

# 3.Pre-processing

In [ ]:
# For CSF3 (UoM) setting
import platform
system_name = platform.system()
print('system_name: ' + system_name)
if system_name == "Linux":
    flag_summary    = False
    flag_cm_norm    = False
    flag_plot_model = False

## Read files

In [ ]:
if not 'X_train' in locals():
    # Load data
    X_train, y_train, X_test, y_test = load_dataset()

    # Split into train and val (No shuffle)
    X_train, X_val, y_train, y_val = \
        train_test_split(X_train, y_train,
        train_size   = ratio_train,
        random_state = seed,
        shuffle      = False)

In [ ]:
# For evaluation
y_test_classes = y_test.argmax(axis=-1)

num_timesteps = X_train.shape[1]
num_features = X_train.shape[2]
num_classes = y_train.shape[1]

cnn_lstm_length = int(num_timesteps / cnn_lstm_steps)
convlstm_length = int(num_timesteps / convlstm_steps)

# For confusion_matrix
labels = np.arange(0, num_classes)

# Input shape of NNs
cnn_1d_input_shape = (num_timesteps, num_features)
lstm_input_shape = (num_timesteps, num_features)
cnn1d_lstm_input_shape = (None, cnn_lstm_length, num_features)
convlstm_input_shape = (convlstm_steps, 1, convlstm_length, num_features)

## (Distributions)

In [ ]:
if flag_data_dist:
    # Remove overlap
    cut = int(X_train.shape[1] / 2)
    X_train_long = X_train[:, -cut:, :]

    # Flatten windows
    X_train_long = X_train_long.reshape((X_train_long.shape[0] * X_train_long.shape[1], X_train_long.shape[2]))

    plot_variable_distributions(X_train_long, start=0, end=8)

## Scaling


In [ ]:
if flag_scaling == "Norm":
    scaler = MinMaxScaler()
elif flag_scaling == "Std":
    scaler = StandardScaler()

# Remove overlap
cut = int(X_train.shape[1] / 2)
X_train_long = X_train[:, -cut:, :]

# Flatten windows
X_train_long = X_train_long.reshape((X_train_long.shape[0] * X_train_long.shape[1], X_train_long.shape[2]))

# Flatten train, val and test
X_train_flat = X_train.reshape((X_train.shape[0] * X_train.shape[1], X_train.shape[2]))
X_val_flat = X_val.reshape((X_val.shape[0] * X_val.shape[1], X_val.shape[2]))
X_test_flat = X_test.reshape((X_test.shape[0] * X_test.shape[1], X_test.shape[2]))

# Fit on training data
scaler.fit(X_train_long)

# Apply to train, val and test
X_train_flat = scaler.transform(X_train_flat)
X_val_flat = scaler.transform(X_val_flat)
X_test_flat = scaler.transform(X_test_flat)

# Reshape
X_train = X_train_flat.reshape((X_train.shape))
X_val = X_val_flat.reshape((X_val.shape))
X_test = X_test_flat.reshape((X_test.shape))

## (Distributions)

In [ ]:
if flag_data_dist:
    # Remove overlap
    cut = int(X_train.shape[1] / 2)
    X_train_long = X_train[:, -cut:, :]

    # Flatten windows
    X_train_long = X_train_long.reshape((X_train_long.shape[0] * X_train_long.shape[1], X_train_long.shape[2]))

    plot_variable_distributions(X_train_long, start=0, end=8)

## [Summary]

In [ ]:
if flag_summary:
    # The number of samples (train, val, test)
    num_samples = len(X_train) + len(X_val) + len(X_test)
    num_train = X_train.shape[0]
    num_val = X_val.shape[0]
    num_test = X_test.shape[0]

    num_classes_train = y_train.shape[-1]
    num_classes_val = y_val.shape[-1]
    num_classes_test = y_test.shape[-1]

    y_counts = pd.concat([np.flip(pd.DataFrame(y_train).value_counts(sort=False)),
                        np.flip(pd.DataFrame(y_val).value_counts(sort=False)),
                        np.flip(pd.DataFrame(y_test).value_counts(sort=False))],
                        axis=1)

    y_counts = y_counts.style.hide_index().highlight_null('red').set_precision(0)

    print('[# of samples]')
    print(f'Total: {num_samples:>7,}')
    print(f'Train: {num_train:>7,}')
    print(f'Val:   {num_val:>7,}')
    print(f'Test:  {num_test:>7,}')
    print()

    print('[# of features]')
    print(f'{num_features}')
    print()

    print('[# of timesteps]')
    print(f'{num_timesteps}')
    print()

    print('[# of classes]')
    print(f'Total: {num_classes}')
    print(f'Train: {num_classes_train}')
    print(f'Val:   {num_classes_val}')
    print(f'Test:  {num_classes_test}')
    print()

    print('[Original data]')
    print('Train   Val   Test')
    display(y_counts)
    print()

# 4.Train & Test


In [ ]:
## callbacks
cb = []

# EarlyStopping
if flag_EarlyStopping:
    es = EarlyStopping(monitor=flag_es_monitor, patience=es_patience)
    cb.append(es)

# TensorBoard
if flag_TensorBoard:
    log_dir = dir_log + '/tb/' + datetime.utcnow().strftime('%Y%m%d-%H%M%S')
    tb = TensorBoard(log_dir=log_dir, histogram_freq=1)
    cb.append(tb)

## CNN


In [ ]:
if flag_CNN_1d:
    if flag_model_load:
        model_cnn_1d = load_model(dir_model + "/" + modelname_cnn_1d + ".h5")
    else:
        model_cnn_1d = build_model_cnn_1d()
    model_cnn_1d.summary()

    if flag_plot_model:
        plot_model(model_cnn_1d, show_shapes=True, to_file='model_' + model_cnn_1d.name + '.png')

In [ ]:
if flag_CNN_1d:
    if not flag_model_load:
        time_start = time.perf_counter()

        history_cnn_1d = model_cnn_1d.fit(
            x               = X_train,
            y               = y_train,
            batch_size      = batch_size,
            epochs          = epochs,
            verbose         = 1,
            callbacks       = [cb],
            validation_data = (X_val, y_val)
        )

        print_execution_time(time_start)

In [ ]:
if flag_CNN_1d:
    if not flag_model_load:
        plot_acc_graph(history_cnn_1d)

In [ ]:
if flag_CNN_1d:
    if flag_model_save:
        time_current = datetime.utcnow().strftime('%Y%m%d-%H%M%S')
        model_cnn_1d.save(dir_model + '/' + model_cnn_1d.name + '_' + time_current + ".h5", overwrite=False)

In [ ]:
if flag_CNN_1d:
    y_pred_classes = model_cnn_1d.predict(X_test).argmax(axis=-1)
    f1 = f1_score(y_test_classes, y_pred_classes, average='weighted')

    print(f'F1 score(weighted) {f1:.4f}\n')
    print(classification_report(y_test_classes, y_pred_classes, target_names=labels_activity))

    if flag_cm_norm:
        cm = confusion_matrix(y_test_classes, y_pred_classes, labels=labels, normalize='true')
        cm = cm * 100
    else:
        cm = confusion_matrix(y_test_classes, y_pred_classes, labels=labels)

    df_cm = pd.DataFrame(cm, index=labels_activity, columns=labels_activity)
    plt.figure(figsize=(5, 5))
    sns.heatmap(df_cm, square=True, annot=True, cbar=False, fmt='.0f', cmap='Greens')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    
    if flag_savefig:
    	plt.savefig("confusion_matrix_" + model_cnn_1d.name + ".png")

    plt.plot()

## LSTM

In [ ]:
if flag_LSTM_Mto1:
    if flag_model_load:
        model_lstm_Mto1 = load_model(dir_model + "/" + modelname_lstm_Mto1 + ".h5")
    else:
        model_lstm_Mto1 = build_model_lstm_Mto1()
    model_lstm_Mto1.summary()

    if flag_plot_model:
        plot_model(model_lstm_Mto1, show_shapes=True, to_file='model_' + model_lstm_Mto1.name + '.png')

In [ ]:
if flag_LSTM_Mto1:
    if not flag_model_load:
        time_start = time.perf_counter()

        history_lstm_Mto1 = model_lstm_Mto1.fit(
            X_train,
            y_train,
            batch_size      = batch_size,
            epochs          = epochs,
            verbose         = 1,
            callbacks       = [cb],
            validation_data = (X_val, y_val)
        )

        print_execution_time(time_start)

In [ ]:
if flag_LSTM_Mto1:
    if not flag_model_load:
        plot_acc_graph(history_lstm_Mto1)

In [ ]:
if flag_LSTM_Mto1:
    if flag_model_save:
        time_current = datetime.utcnow().strftime('%Y%m%d-%H%M%S')
        model_lstm_Mto1.save(dir_model + '/' + model_lstm_Mto1.name + '_' + time_current + ".h5", overwrite=False)

In [ ]:
if flag_LSTM_Mto1:
    y_pred_classes = model_lstm_Mto1.predict(X_test).argmax(axis=-1)
    f1 = f1_score(y_test_classes, y_pred_classes, average='weighted')

    print(f'F1 score(weighted) {f1:.4f}\n')
    print(classification_report(y_test_classes, y_pred_classes, target_names=labels_activity))

    if flag_cm_norm:
        cm = confusion_matrix(y_test_classes, y_pred_classes, labels=labels, normalize='true')
        cm = cm * 100
    else:
        cm = confusion_matrix(y_test_classes, y_pred_classes, labels=labels)

    df_cm = pd.DataFrame(cm, index=labels_activity, columns=labels_activity)
    plt.figure(figsize=(5, 5))
    sns.heatmap(df_cm, square=True, annot=True, cbar=False, fmt='.0f', cmap='Greens')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    
    if flag_savefig:
    	plt.savefig("confusion_matrix_" + model_lstm_Mto1.name + ".png")

    plt.plot()

## CNN-LSTM

In [ ]:
# Reshape data into 4D for CNN1D-LSTM
X_train_cnn1d_lstm = X_train.reshape((X_train.shape[0], cnn_lstm_steps, cnn_lstm_length, X_train.shape[-1]))
X_val_cnn1d_lstm = X_val.reshape((X_val.shape[0], cnn_lstm_steps, cnn_lstm_length, X_val.shape[-1]))
X_test_cnn1d_lstm = X_test.reshape((X_test.shape[0], cnn_lstm_steps, cnn_lstm_length, X_test.shape[-1]))

In [ ]:
if flag_CNN1D_LSTM:
    if flag_model_load:
        model_cnn1d_lstm = load_model(dir_model + "/" + modelname_cnn1d_lstm + ".h5")
    else:
        model_cnn1d_lstm = build_model_cnn1d_lstm()
    model_cnn1d_lstm.summary()

    if flag_plot_model:
        plot_model(model_cnn1d_lstm, show_shapes=True, to_file='model_' + model_cnn1d_lstm.name + '.png')

In [ ]:
if flag_CNN1D_LSTM:
    if not flag_model_load:
        time_start = time.perf_counter()

        history_cnn1d_lstm = model_cnn1d_lstm.fit(
            X_train_cnn1d_lstm,
            y_train,
            batch_size      = batch_size,
            epochs          = epochs,
            verbose         = 1,
            callbacks       = [cb],
            validation_data = (X_val_cnn1d_lstm, y_val)
        )

        print_execution_time(time_start)

In [ ]:
if flag_CNN1D_LSTM:
    if not flag_model_load:
        plot_acc_graph(history_cnn1d_lstm)

In [ ]:
if flag_CNN1D_LSTM:
    if flag_model_save:
        time_current = datetime.utcnow().strftime('%Y%m%d-%H%M%S')
        model_cnn1d_lstm.save(dir_model + '/' + model_cnn1d_lstm.name + '_' + time_current + ".h5", overwrite=False)

In [ ]:
if flag_CNN1D_LSTM:
    y_pred_classes = model_cnn1d_lstm.predict(X_test_cnn1d_lstm).argmax(axis=-1)
    f1 = f1_score(y_test_classes, y_pred_classes, average='weighted')

    print(f'F1 score(weighted) {f1:.4f}\n')
    print(classification_report(y_test_classes, y_pred_classes, target_names=labels_activity))

    if flag_cm_norm:
        cm = confusion_matrix(y_test_classes, y_pred_classes, labels=labels, normalize='true')
        cm = cm * 100
    else:
        cm = confusion_matrix(y_test_classes, y_pred_classes, labels=labels)

    df_cm = pd.DataFrame(cm, index=labels_activity, columns=labels_activity)
    plt.figure(figsize=(5, 5))
    sns.heatmap(df_cm, square=True, annot=True, cbar=False, fmt='.0f', cmap='Greens')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    
    if flag_savefig:
    	plt.savefig("confusion_matrix_" + model_cnn1d_lstm.name + ".png")

    plt.plot()

## ConvLSTM

In [ ]:
# Reshape data into 5D for ConvLSTM
X_train_convlstm = X_train.reshape((X_train.shape[0], convlstm_steps, 1, convlstm_length, X_train.shape[-1]))
X_val_convlstm = X_val.reshape((X_val.shape[0], convlstm_steps, 1, convlstm_length, X_val.shape[-1]))
X_test_convlstm = X_test.reshape((X_test.shape[0], convlstm_steps, 1, convlstm_length, X_test.shape[-1]))

In [ ]:
if flag_ConvLSTM:
    if flag_model_load:
        model_convlstm = load_model(dir_model + "/" + modelname_convlstm + ".h5")
    else:
        model_convlstm = build_model_convlstm()
    model_convlstm.summary()

    if flag_plot_model:
        plot_model(model_convlstm, show_shapes=True, to_file='model_' + model_convlstm.name + '.png')

In [ ]:
if flag_ConvLSTM:
    if not flag_model_load:
        time_start = time.perf_counter()

        history_convlstm = model_convlstm.fit(
            X_train_convlstm,
            y_train,
            batch_size      = batch_size,
            epochs          = epochs,
            verbose         = 1,
            callbacks       = [cb],
            validation_data = (X_val_convlstm, y_val)
        )

        print_execution_time(time_start)

In [ ]:
if flag_ConvLSTM:
    if not flag_model_load:
        plot_acc_graph(history_convlstm)

In [ ]:
if flag_ConvLSTM:
    if flag_model_save:
        time_current = datetime.utcnow().strftime('%Y%m%d-%H%M%S')
        model_convlstm.save(dir_model + '/' + model_convlstm.name + '_' + time_current + ".h5", overwrite=False)

In [ ]:
if flag_ConvLSTM:
    y_pred_classes = model_convlstm.predict(X_test_convlstm).argmax(axis=-1)
    f1 = f1_score(y_test_classes, y_pred_classes, average='weighted')

    print(f'F1 score(weighted) {f1:.4f}\n')
    print(classification_report(y_test_classes, y_pred_classes, target_names=labels_activity))

    if flag_cm_norm:
        cm = confusion_matrix(y_test_classes, y_pred_classes, labels=labels, normalize='true')
        cm = cm * 100
    else:
        cm = confusion_matrix(y_test_classes, y_pred_classes, labels=labels)

    df_cm = pd.DataFrame(cm, index=labels_activity, columns=labels_activity)
    plt.figure(figsize=(5, 5))
    sns.heatmap(df_cm, square=True, annot=True, cbar=False, fmt='.0f', cmap='Greens')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    
    if flag_savefig:
    	plt.savefig("confusion_matrix_" + model_convlstm.name + ".png")

    plt.plot()

## Ensemble

In [ ]:
if flag_Ensemble:
    # build sub-models
    sub_models = list()
    X_train_ensemble = list()
    X_val_ensemble = list()
    X_test_ensemble = list()

    if flag_CNN_1d:
        sub_models.append(model_cnn_1d)
        X_train_ensemble.append(X_train)
        X_val_ensemble.append(X_val)
        X_test_ensemble.append(X_test)
    if flag_LSTM_Mto1:
        sub_models.append(model_lstm_Mto1)
        X_train_ensemble.append(X_train)
        X_val_ensemble.append(X_val)
        X_test_ensemble.append(X_test)
    if flag_CNN1D_LSTM:
        sub_models.append(model_cnn1d_lstm)
        X_train_ensemble.append(X_train_cnn1d_lstm)
        X_val_ensemble.append(X_val_cnn1d_lstm)
        X_test_ensemble.append(X_test_cnn1d_lstm)
    if flag_ConvLSTM:
        sub_models.append(model_convlstm)
        X_train_ensemble.append(X_train_convlstm)
        X_val_ensemble.append(X_val_convlstm)
        X_test_ensemble.append(X_test_convlstm)

    # freeze parameters for sub-models
    for i in range(len(sub_models)):
        for layer in sub_models[i].layers:
            layer.trainable = False
            layer._name = 'ensemble_' + sub_models[i].name + '_' + layer.name

    inputs = [model.input for model in sub_models]
    outputs = [model.output for model in sub_models]

    model_ensemble = build_model_ensemble(inputs, outputs)
    model_ensemble.summary()

    if flag_plot_model:
        plot_model(model_ensemble, show_shapes=True, to_file='model_' + model_ensemble.name + '.png')

In [ ]:
if flag_Ensemble:
    history_ensemble    = model_ensemble.fit(
        x               = X_train_ensemble,
        y               = y_train,
        batch_size      = batch_size,
        epochs          = epochs,
        verbose         = 1,
        callbacks       = [cb],
        validation_data = (X_val_ensemble, y_val)
    )

In [ ]:
if flag_Ensemble:
    plot_acc_graph(history_ensemble)

In [ ]:
if flag_Ensemble:
    y_pred_classes = model_ensemble.predict(X_test_ensemble).argmax(axis=-1)
    f1 = f1_score(y_test_classes, y_pred_classes, average='weighted')

    print(f'F1 score(weighted) {f1:.4f}\n')
    print(classification_report(y_test_classes, y_pred_classes, target_names=labels_activity))

    if flag_cm_norm:
        cm = confusion_matrix(y_test_classes, y_pred_classes, labels=labels, normalize='true')
        cm = cm * 100
    else:
        cm = confusion_matrix(y_test_classes, y_pred_classes, labels=labels)

    df_cm = pd.DataFrame(cm, index=labels_activity, columns=labels_activity)
    plt.figure(figsize=(5, 5))
    sns.heatmap(df_cm, square=True, annot=True, cbar=False, fmt='.0f', cmap='Greens')
    plt.xlabel('Predicted')
    plt.ylabel('True')

    if flag_savefig:
        plt.savefig("confusion_matrix_" + model_ensemble.name + ".png")

    plt.plot()

# 5.Experiment

In [ ]:
if flag_experiment:
    time_start = time.perf_counter()

    print(f'repeats: {repeats}, epochs: {epochs}, batch_size: {batch_size}')
    if flag_EarlyStopping:
        print(f' EarlyStopping is set (patience: {es_patience})\n')
    else:
        print(f' no EarlyStopping\n')

    ## CNN 1D
    run_experiment(modelname_cnn_1d,
                   X_train, X_val, X_test,
                   y_train, y_val, y_test,
                   repeats=repeats)

    ## LSTM Mto1 (stateless)
    run_experiment(modelname_lstm_Mto1,
                   X_train, X_val, X_test,
                   y_train, y_val, y_test,
                   repeats=repeats)

    ## CNN1D-LSTM
    run_experiment(modelname_cnn1d_lstm,
                   X_train_cnn1d_lstm, X_val_cnn1d_lstm, X_test_cnn1d_lstm,
                   y_train, y_val, y_test,
                   repeats=repeats)

    ## ConvLSTM
    run_experiment(modelname_convlstm,
                   X_train_convlstm, X_val_convlstm, X_test_convlstm,
                   y_train, y_val, y_test,
                   repeats=repeats)
    
    print_execution_time(time_start)